<a href="https://colab.research.google.com/github/texaspandaa/NZ/blob/main/topicmodelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 37.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
%matplotlib inline
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from google.colab import drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**load data**

In [ ]:
drive.mount('drive', True)

In [ ]:
data = pd.read_csv('drive/My Drive/NZ/fackcheck headline data/headlines.csv')

In [ ]:
data.head(6)

,Unnamed: 0,date,statement,rating,fact_check,harm_PC1,veracity
0,1,"December 14, 2022",Grant Wahl died because of the COVID-19 vaccine.,false,politifact,-0.525641,False
1,2,"December 14, 2022","Sen. Kyrsten Sinema ""voted with the president ...",mostly-true,politifact,-1.126343,True
2,3,"December 14, 2022","“For the last several months, wages have gone ...",mostly-true,politifact,-1.146951,True
3,4,"December 14, 2022",The Federal Reserve “has suspended the availab...,false,politifact,-1.052610,False
4,7,"December 13, 2022","Large machinery is “required to move 500,000 p...",false,politifact,-1.110309,False
5,8,"December 13, 2022",The war in Ukraine is “all over.”,false,politifact,-0.405668,False


In [ ]:
data_statement = data['statement']

 **Tokenization**

In [ ]:
data_statement_p = data_statement.map(lambda x:gensim.utils.simple_preprocess(x, deacc=True))
# print(type(data_statement_p))
# data_statement_p.head(3)
data_statement_p = list(data_statement_p)
print(data_statement_p[0])

[['grant', 'wahl', 'died', 'because', 'of', 'the', 'covid', 'vaccine'],
 ['sen',
  'kyrsten',
  'sinema',
  'voted',
  'with',
  'the',
  'president',
  'of',
  'the',
  'time'],
 ['for',
  'the',
  'last',
  'several',
  'months',
  'wages',
  'have',
  'gone',
  'up',
  'more',
  'than',
  'prices',
  'have',
  'gone',
  'up'],
 ['the',
  'federal',
  'reserve',
  'has',
  'suspended',
  'the',
  'availability',
  'of',
  'new',
  'currency'],
 ['large',
  'machinery',
  'is',
  'required',
  'to',
  'move',
  'pounds',
  'of',
  'earth',
  'in',
  'order',
  'to',
  'get',
  'the',
  'minerals',
  'needed',
  'for',
  'one',
  'single',
  'tesla',
  'car',
  'battery'],
 ['the', 'war', 'in', 'ukraine', 'is', 'all', 'over'],
 ['this',
  'video',
  'shows',
  'chuck',
  'grassley',
  'demanding',
  'joe',
  'biden',
  'resignation'],
 ['early',
  'vote',
  'counts',
  'after',
  'the',
  'polls',
  'closed',
  'in',
  'georgia',
  'senate',
  'runoff',
  'are',
  'evidence',
  'of',
 

**bigrams and trigrams**

In [ ]:
#bulding bigram and trigram models
bigram = gensim.models.Phrases(data_statement_p, min_count=3)
trigram = gensim.models.Phrases(bigram[data_statement_p], threshold=100)

#get sentence clubbed as bigram trigrams
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# print(trigram_mod[bigram_mod[data_statement_p[1]]])

['sen', 'kyrsten_sinema', 'voted', 'with', 'the', 'president', 'of', 'the', 'time']


**remove stop words**

In [ ]:
stop_words = stopwords.words('english')
data_statement_s = [ [word for word in text if word not in stop_words] for text in data_statement_p]

[['grant', 'wahl', 'died', 'covid', 'vaccine'],
 ['sen', 'kyrsten', 'sinema', 'voted', 'president', 'time'],
 ['last', 'several', 'months', 'wages', 'gone', 'prices', 'gone'],
 ['federal', 'reserve', 'suspended', 'availability', 'new', 'currency'],
 ['large',
  'machinery',
  'required',
  'move',
  'pounds',
  'earth',
  'order',
  'get',
  'minerals',
  'needed',
  'one',
  'single',
  'tesla',
  'car',
  'battery'],
 ['war', 'ukraine'],
 ['video',
  'shows',
  'chuck',
  'grassley',
  'demanding',
  'joe',
  'biden',
  'resignation'],
 ['early',
  'vote',
  'counts',
  'polls',
  'closed',
  'georgia',
  'senate',
  'runoff',
  'evidence',
  'election',
  'fraud'],
 ['recent',
  'blood',
  'clots',
  'strokes',
  'four',
  'major',
  'athletes',
  'unwanted',
  'effect',
  'covid',
  'vaccines'],
 ['human',
  'meat',
  'project',
  'using',
  'donated',
  'bodies',
  'organs',
  'create',
  'alternative',
  'meat',
  'consumption',
  'options'],
 ['jag', 'charges', 'nancy', 'pelosi'

**lemma**